In [1]:
# import section

In [2]:
# Notes section
'''
Capital letter?
'''


'\nCapital letter?\n'

In [3]:
likelihood = {}
transition = {}

In [4]:
# read training/development files

file = open("WSJ_24.pos")
lines = file.read()
file.close()

lines1 = lines.split('\n')

file = open("WSJ_02-21.pos")
lines = file.read()
file.close()

lines2 = lines.split('\n')

lines = lines1 + lines2

In [5]:
# Fill in the likelihood

for line in lines:
    line = line.split("\t")
    '''
    print(line[0])
    if len(line) == 1:
        print(0)
    else:
        print(1)
    '''
    if len(line) > 1:
        # This line is not a blank line
        if line[1] in likelihood:
            if line[0] in likelihood[line[1]]:
                likelihood[line[1]][line[0]] += 1
            else:
                likelihood[line[1]][line[0]] = 1
        else:
            likelihood[line[1]] = {line[0]: 1}
    else:
        # This line is a blank line
        pass

'''
print(len(likelihood))
print(likelihood['NN'])
for i in likelihood:
    print(i)
'''


"\nprint(len(likelihood))\nprint(likelihood['NN'])\nfor i in likelihood:\n    print(i)\n"

In [6]:
# Collect all states
states = []
states_full = ['begin_sentence', 'end_sentence']
for key in likelihood:
    states.append(key)
    states_full.append(key)
#print(states)
#print(states_full)

In [7]:
# Collect all words
words = []
for key in likelihood:
    for word in likelihood[key]:
        if word not in words:
            words.append(word)
        else:
            pass


In [8]:
# Count words

words_count = {}
for word in words:
    words_count[word] = [0, '']
    
for line in lines:
    line = line.split("\t")
    if len(line) > 1:
        # This line is not a blank line
        words_count[line[0]][0] += 1
        words_count[line[0]][1] = line[1]
#print(words_count)#

In [9]:
# From the result of counting words, find distribution of OOV
OOV = {}
sum = 0
for state in states:
    OOV[state] = 0
for word in words_count:
    if words_count[word][0] == 1:
        OOV[words_count[word][1]] += 1
        sum += 1
#print(OOV)#
for state in OOV:
    OOV[state] /= sum
print(OOV)#

{'DT': 9.548818333731201e-05, 'NN': 0.16280735259011697, 'POS': 0.0, 'MD': 0.000143232275005968, 'VB': 0.02282167581761757, 'VBN': 0.027643829076151826, 'IN': 0.0008116495583671521, 'JJ': 0.1892575793745524, 'NNS': 0.09505848651229411, ',': 0.0, 'CC': 4.7744091668656006e-05, '.': 0.0, 'RBS': 4.7744091668656006e-05, 'NNP': 0.24282645022678442, 'VBZ': 0.018715683934113155, 'TO': 0.0, '$': 4.7744091668656006e-05, 'CD': 0.14122702315588445, 'VBG': 0.03638099785151588, 'RB': 0.020959656242539985, 'VBD': 0.016901408450704224, 'PRP': 0.00047744091668656003, 'PRP$': 4.7744091668656006e-05, 'VBP': 0.0048221532585342565, 'WRB': 0.0, '``': 4.7744091668656006e-05, 'EX': 0.0, "''": 0.0, ':': 0.0, 'RBR': 0.00023872045834328001, 'WP': 0.0, 'JJR': 0.0032943423251372644, 'WDT': 9.548818333731201e-05, 'RP': 0.0, '#': 0.0, 'PDT': 0.0, 'UH': 0.0008116495583671521, '(': 0.0, ')': 0.0, 'NNPS': 0.009596562425399857, 'JJS': 0.0021484841250895203, 'SYM': 4.7744091668656006e-05, 'FW': 0.0025781809501074244, 'WP

In [10]:
# Fill in the transition
previous_state = 'begin_sentence'
for line in lines:
    line = line.split("\t")
    # line[0] is word
    # line[1] is category, or current state
    if len(line) > 1:
        # This line is not a blank line
        if previous_state in transition:
            if line[1] in transition[previous_state]:
                transition[previous_state][line[1]] += 1
            else:
                transition[previous_state][line[1]] = 1
        else:
            transition[previous_state] = {line[1]: 1}
        previous_state = line[1]
    else:
        # This line is a blank line
        if previous_state in transition:
            if 'end_sentence' in transition[previous_state]:
                transition[previous_state]['end_sentence'] += 1
            else:
                transition[previous_state]['end_sentence'] = 1
        else:
            transition[previous_state] = {'end_sentence': 1}
        previous_state = 'begin_sentence'

'''
print(len(transition))
print(transition['NN'])
for i in transition:
    print(i)
'''


"\nprint(len(transition))\nprint(transition['NN'])\nfor i in transition:\n    print(i)\n"

In [11]:
# Change frequencies to probabilities for likelihood and transition
for i in likelihood:
    sum = 0
    for j in likelihood[i]:
        sum += likelihood[i][j]
    for j in likelihood[i]:
        likelihood[i][j] /= sum

for i in transition:
    sum = 0
    for j in transition[i]:
        sum += transition[i][j]
    for j in transition[i]:
        transition[i][j] /= sum
#print(transition['NN'])

In [24]:
# read test files

file = open("WSJ_23.words")
#file = open("debug_test.words")#
lines = file.read()
file.close()

lines = lines.split('\n')

In [25]:
# Viterbi algorithm for HMM

f = open("submission.pos", "w")

observed_words = []

try:
    for line in lines:
        '''
        print(line)
        if line == '':
            print(0)
        else:
            print(1)
        '''
        # One sentence finishes
        if line == '':
            # For viterbi and backpointer matrix, dimension is len(states)+2 * len(observed_words)
            viterbi = [[0 for _ in range(len(observed_words))] for _ in range(len(states) + 2)]
            backpointer = [[-1 for _ in range(len(observed_words))] for _ in range(len(states) + 2)]
            for q in range(len(states)):
                # q is index of state
                if observed_words[0] in words:
                    # is not an OOV word
                    try:
                        part1 = transition['begin_sentence'][states[q]]
                        #print('test point')#
                    except:
                        part1 = 0
                    try:
                        part2 = likelihood[states[q]][observed_words[0]]
                        #print('test point')#
                    except:
                        part2 = 0
                    viterbi[q][0] = part1 * part2
                    backpointer[q][0] = len(states)
                    # len(states) in above line is index of second last position of backpointer matrix
                    # second last position is for begin_sentence and last position is for end_sentence
                else:
                    # is an OOV word
                    viterbi[q][0] = OOV[states[q]]
                    backpointer[q][0] = len(states)
            for w in range(1, len(observed_words)):
                for q in range(len(states)):
                    score_max = float('-inf')
                    pointer_max = -1
                    for q_previous in range(len(states)):
                        if observed_words[w] in words:
                            try:
                                part1 = transition[states[q_previous]][states[q]]
                                #print('test point')#
                            except:
                                part1 = 0
                            try:
                                part2 = likelihood[states[q]][observed_words[w]]
                                #print('test point')#
                            except:
                                part2 = 0
                            score = viterbi[q_previous][w-1] * part1 * part2
                        else:
                            score = viterbi[q_previous][w-1] * OOV[states[q]]
                        if score > score_max:
                            score_max = score
                            pointer_max = q_previous
                    # end for, now find out max for score and pointer
                    viterbi[q][w] = score_max
                    backpointer[q][w] = pointer_max
            # Set end of sentence
            score_max = float('-inf')
            pointer_max = -1
            for q in range(len(states)):
                try:
                    part1 = transition[states[q]]['end_sentence']
                except:
                    part1 = 0
                score = viterbi[q][len(observed_words)-1] * part1
                if score > score_max:
                    score_max = score
                    pointer_max = q
            backpointer[len(states)+1][len(observed_words)-1] = pointer_max
            # retrieve path
            w = len(observed_words)-1
            pointer = backpointer[len(states)+1][w]
            path = [states[pointer]]
            while w > 0:
                pointer = backpointer[pointer][w]
                path.append(states[pointer])
                w -= 1
            path.reverse()
            #print(path)#
            #print(viterbi)#
            #print(backpointer)#
            
            #'''
            for i in range(len(path)):
                f.write(observed_words[i])
                f.write('\t')
                f.write(path[i])
                f.write('\n')
            f.write('\n')
            #'''

            observed_words = []
        # Current sentence continues
        else:
            observed_words.append(line)
except:
    print('It shows an error occurs, but the file has been fully written already. The error will cause no impact.')

f.close()

It shows an error occurs, but the file has been fully written already. The error will cause no impact.
